<a href="https://colab.research.google.com/github/18K41A0536/NLP-Project/blob/main/Major_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd

# dataset coronavirus WHO
pd.set_option('max_colwidth', 100)  # Increase column width
data = pd.read_excel("Covid-19 Information.xlsx")
data.head()

,Context,Answer
0,What is a coronavirus?,Coronaviruses are a large family of viruses which may cause illness in animals or humans.
1,What is a coronavirus?,"In humans, several coronaviruses are known to cause respiratory infections ranging from the comm..."
2,What is COVID-19?,COVID-19 is the infectious disease caused by the most recently discovered coronavirus. This new ...
3,What are the symptoms of COVID-19?,"The most common symptoms of COVID-19 are fever, tiredness, and dry cough. Some patients may have..."
4,What are the symptoms of COVID-19?,Some people become infected but don’t develop any symptoms and don't feel unwell. Most people (a...


In [12]:
!pip install tensorflow_text

     |████████████████████████████████| 4.4 MB 5.4 MB/s 


In [13]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import re

def preprocess_sentences(input_sentences):
    return [re.sub(r'(covid-19|covid)', 'coronavirus', input_sentence, flags=re.I) 
            for input_sentence in input_sentences]
        
# Load module containing USE
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

# Create response embeddings
response_encodings = module.signatures['response_encoder'](
        input=tf.constant(preprocess_sentences(data.Answer)),
        context=tf.constant(preprocess_sentences(data.Context)))['outputs']

In [14]:
test_questions = [
    "What about pregnant women?",
    "Wat is de lengte van de incubatietijd?",
    "Are animals contagious COVID-19?",
    "Are there medicine against the coronavirus?",
    "Can I breastfead when I have COVID-19?",
    "Should I stay inside the house?",  # English questions are also possible.
    "Kann ich mit meinem Hund spazieren gehen?"  # As well as German, and all the other languages supported by use-multilingual.
]

# Create encodings for test questions
question_encodings = module.signatures['question_encoder'](
    tf.constant(preprocess_sentences(test_questions))
)['outputs']

# Get the responses
test_responses = data.Answer[np.argmax(np.inner(question_encodings, response_encodings), axis=1)]

# Show them in a dataframe
pd.DataFrame({'Test Questions': test_questions, 'Test Responses': test_responses})

,Test Questions,Test Responses
65,What about pregnant women?,"due to changes in their bodies and immune systems, we know that pregnant women can be badly affe..."
44,Wat is de lengte van de incubatietijd?,"Most estimates of the incubation period for COVID-19 range from 1-14 days, most commonly around ..."
45,Are animals contagious COVID-19?,"Coronaviruses are a large family of viruses that are common in animals. Occasionally, people get..."
32,Are there medicine against the coronavirus?,"Not yet. To date, there is no vaccine and no specific antiviral medicine to prevent or treat COV..."
72,Can I breastfead when I have COVID-19?,Yes. Women with COVID-19 can breastfeed if they wish to do so.
17,Should I stay inside the house?,"Stay home if you feel unwell. If you have a fever, cough and difficulty breathing, seek medical ..."
18,Kann ich mit meinem Hund spazieren gehen?,"If possible, avoid traveling to places – especially if you are an older person or have diabetes..."
